Extract Frames script

In [2]:
import os
import subprocess

In [4]:
def extract_frames(src_vid_dir: str, dest_frames_dir: str, frames_per_vid: int = 200):
    # Create the output folder if it doesn't exist
    os.makedirs(src_vid_dir, exist_ok=True)

    # Get video list
    vid_list = os.listdir(src_vid_dir)
    num_vids = len(vid_list)

    # Iterate over videos
    count = 1
    for vid_name in vid_list:
        print(f"{vid_name} ({count} of {num_vids})")

        # Check if the file is a video (you can extend this check based on formats)
        vid_path = os.path.join(src_vid_dir, vid_name)
        if os.path.isfile(vid_path) and vid_name.lower().endswith(('.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv')):
            # Extract file name without extension
            name, _ = os.path.splitext(vid_name)

            # Create a subfolder for frames of this video
            video_output_folder = os.path.join(dest_frames_dir, name)
            os.makedirs(video_output_folder, exist_ok=True)

            # Get the video duration using ffprobe
            try:
                result = subprocess.run(
                    ["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "csv=p=0", vid_path],
                    capture_output=True, text=True, check=True
                )
                duration = float(result.stdout.strip())
            except subprocess.CalledProcessError as e:
                print(f"Error retrieving duration for {vid_name}: {e}")
                continue

            # Calculate the interval between frames (in seconds)
            frame_interval = duration / frames_per_vid

            # Build the ffmpeg command
            command = [
                "ffmpeg",
                "-i", vid_path,
                "-vf", f"fps=1/{frame_interval}",
                "-vsync", "vfr",
                os.path.join(video_output_folder, "frame_%04d.png")
            ]

            # Run the ffmpeg command
            try:
                print(f"Extracting frames from {vid_name}...")
                subprocess.run(command, check=True)
                print(f"Frames saved to {video_output_folder}")
            except subprocess.CalledProcessError as e:
                print(f"Error processing {vid_name}: {e}")
                
        count += 1

In [ ]:
input_folder = "train_videos"
output_folder = "train_frames"
frames_per_vid = 200

